<a href="https://colab.research.google.com/github/aleklearn/repo/blob/main/generative-ai/mixtral_8x7b_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
- [Running Mixtral 8x7b On Google Colab For Free](https://www.kdnuggets.com/running-mixtral-8x7b-on-google-colab-for-free)
- [Mixtral of Experts](https://mistral.ai/news/mixtral-of-experts/), [paper](https://arxiv.org/pdf/2401.04088.pdf)
- [LLaMA C++](https://github.com/ggerganov/llama.cpp)


### Install `llama.cpp`

In [ ]:
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


In [ ]:
%cd llama.cpp

!make LLAMA_CUBLAS=1

/content/llama.cpp
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-

### Mixtral 8x7B Model

https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/blob/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf


In [ ]:
!pip install -Uq huggingface-hub

In [ ]:
!huggingface-cli download TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF mixtral-8x7b-instruct-v0.1.Q2_K.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf to /root/.cache/huggingface/hub/tmplnqswpvi
mixtral-8x7b-instruct-v0.1.Q2_K.gguf: 100% 15.6G/15.6G [02:03<00:00, 126MB/s]
./mixtral-8x7b-instruct-v0.1.Q2_K.gguf


### Running the Server

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(6589)"))

https://nb0vw35rmp-496ff2e9c6d22116-6589-colab.googleusercontent.com/


- `-m FNAME`, `--model FNAME`: Specify the path to the LLaMA model file (e.g., `models/7B/ggml-model.gguf`).
- `-ngl N`, `--n-gpu-layers N`: When compiled with appropriate support (currently CLBlast or cuBLAS), this option allows offloading some layers to the GPU for computation. Generally results in increased performance.
- `-c N`, `--ctx-size N`: Set the size of the prompt context. The default is 512, but LLaMA models were built with a context of 2048, which will provide better results for longer input/inference. The size may differ in other models, for example, baichuan models were build with a context of 4096.
- `--port`: Set the port to listen. Default: `8080`.

[More details](https://github.com/ggerganov/llama.cpp/tree/master/examples/server)

In [ ]:
%cd /content/llama.cpp

!./server -m mixtral-8x7b-instruct-v0.1.Q2_K.gguf -ngl 27 -c 2048 --port 6589

/content/llama.cpp
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
{"timestamp":1705463346,"level":"INFO","function":"main","line":2864,"message":"build info","build":1,"commit":"5c99960"}
{"timestamp":1705463346,"level":"INFO","function":"main","line":2867,"message":"system info","n_threads":2,"n_threads_batch":-1,"total_threads":2,"system_info":"AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | "}

llama server listening at http://127.0.0.1:6589

{"timestamp":1705463346,"level":"INFO","function":"main","line":2971,"message":"HTTP server listening","port":"6589","hostname":"127.0.0.1"}
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from mixtral-8x7b-instruct-v0.1.Q2_K.g